In [ ]:
import re


pattern = r'@(\w+)\{((?:[^{}]*|\{(?:[^{}]*|\{[^{}]*\})*\})*)\}'

with open('bib/references.bib','r') as fdata:
    text = fdata.read()

import bibtexparser
bibs = bibtexparser.parse_string(text)
import json 
# json.dumps(bibs.entries_dict,indent=4)

for name,entry in bibs.entries_dict.items():
    print(entry)

with open("webcrawl-data.json", 'r') as fdata:
    data = json.load(fdata)

print(len(data))

import requests

def get_doi_bibtex(doi):
    base_url = f"https://doi.org/{doi}"
    headers = {
        "Accept": "application/citeproc+json"
    }
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:
        return response.text.strip()
    else:
        print(response.status_code)
        print(response.text)
        return None
    
def jsonToEntry(data):
    try: 
        obj = {
            "id": data["author"][0]["family"] + str(data["published"]["date-parts"][0][0]),
            "title": data["title"],
            "booktitle": data["container-title"], 
            "doi": data["DOI"],   
            "citation_count": data["is-referenced-by-count"], 
            "keywords": "None"
        }
        return obj
    except: 
        return None

import tqdm
import time
new_data = list()
for entry in tqdm.tqdm(data):
    if entry['id'] == "unknown":
        doiData = get_doi_bibtex(entry['doi'])
        if not doiData: continue 
        doiData = json.loads(doiData)
        
        with open("test.json", 'w') as fdata:
            json.dump(doiData,fdata,indent=4)

        newEntry = jsonToEntry(doiData)
        if newEntry:
            new_data.append(newEntry)
        time.sleep(0.1)


data = [e for e in data if e['id'] != "unknown"]
data.extend(new_data)

with open("updated-crawl.json", 'w') as fdata:
    json.dump(data,fdata,indent=4)


In [1]:
import json
with open("updated-crawl.json") as fdata:
    data = json.load(fdata)

keywords = {"graph", "network", "mds", "scaling", "dimension", "som", "layout", "embed", "drawing"}

def check_substrings_in_string(keywords,word):
    for keyword in keywords:
        if keyword in word: return True 

    return False

toSave = list()
for entry in data:
    if check_substrings_in_string(keywords,entry['title'].lower()):
        toSave.append(entry)

In [3]:
len(toSave)

1615

In [4]:
import requests

def get_doi_bibtex(doi):
    base_url = f"https://doi.org/{doi}"
    headers = {
        "Accept": "text/bibliography; style=bibtex"
    }
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:
        return response.text.strip()
    else:
        print(response.status_code)
        print(response.text)
        return None

In [5]:
bibitems = [get_doi_bibtex(d['doi']) for d in toSave]

404
<!DOCTYPE html>
<html lang="ko">
<head>
	<meta charset="utf-8">
	<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<title>404 Error</title>

	<!-- HTML5 Shim and Respond.js IE8 support of HTML5 elements and media queries -->
	<!--[if lt IE 9]>
	<script src="/js/html5shiv.min.js"></script>
	<script src="/js/html5shiv-printshiv.min.js"></script>
	<script src="https://oss.maxcdn.com/libs/respond.js/1.3.0/respond.min.js"></script>
	<![endif]-->


	<!--[if lt IE 10]>
	<style type="text/css">.placeholder { color: #aaa; }</style>
	<script src="/jquery/jquery.placeholder.min.js"></script>
	<script>$(function() { $('input, textarea').placeholder(); });</script>
	<![endif]-->

	<link rel="stylesheet" type="text/css" href="//fonts.googleapis.com/css?family=Open+Sans:300,400,600,700">

	<link rel="stylesheet" type="text/css" href="/jquery/toastr/toastr.min.css">
	<link rel="stylesheet" type="text/css" href="/jquery/sweetalert/sweetalert.css">

	<link rel="stylesheet" type

In [26]:
bibitems = [bib for bib in bibitems if bib and "<!DOCTYPE" not in bib]

In [27]:
bibitems[2].encode('latin1').decode('utf-8')

'@article{Kruiger_2017, title={Graph Layouts by t‐SNE}, volume={36}, ISSN={1467-8659}, url={http://dx.doi.org/10.1111/cgf.13187}, DOI={10.1111/cgf.13187}, number={3}, journal={Computer Graphics Forum}, publisher={Wiley}, author={Kruiger, J. F. and Rauber, P. E. and Martins, R. M. and Kerren, A. and Kobourov, S. and Telea, A. C.}, year={2017}, month=jun, pages={283–294} }'

In [28]:
bibitems_newline = [bib.replace("},", "},\n") for bib in bibitems]
bibitems_newline = [bib.replace(",",",\n", 1) for bib in bibitems_newline]
newitems = list()
for bib in bibitems_newline:
    try: 
        item = bib.encode('latin1').decode('utf-8')
        newitems.append(item)
    except:
        newitems.append(bib.encode('utf-8').decode('utf-8'))
bibitems_newline = newitems

In [29]:
with open('references.bib', 'w') as fdata:
    s = "\n\n".join(bibitems_newline)
    fdata.write(s)